In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter
from subprocess import check_output
#看看文件是否存在，并且是不是用utf8
print(check_output(["ls", "input"]).decode("utf8"))

Authorization for Minor Form.rtf
child_wishlist_v2.csv
gift_goodkids_v2.csv
sample_submission_random_v2.csv



### 后面看懂代码后改成dataframe
```python
drop(0,1)的意思是：其中1表示axis=1,0表示名称，在这里即为删除名称为0的列
gift_pref_df = pd.read_csv('../input/child_wishlist_v2.csv',header=None).drop(0,1)
child_pref_df = pd.read_csv('../input/gift_goodkids_v2.csv',header=None).drop(0,1)
```

In [3]:
#居然转换成了ndarry
#孩子喜欢的礼物
gift_pref = pd.read_csv('input/child_wishlist_v2.csv',header=None).drop(0, 1).values
#礼物喜欢的孩子
child_pref = pd.read_csv('input/gift_goodkids_v2.csv',header=None).drop(0, 1).values

In [4]:
type(child_pref)

numpy.ndarray

In [6]:
print(np.min(gift_pref))
print(np.max(gift_pref))

0
999


In [7]:
print(np.min(child_pref))
print(np.max(child_pref))

0
999999


In [5]:
#有1000000个children
n_children = 1000000 
#1000种gift
n_gift_type = 1000
#每种gift1000个
n_gift_quantity = 1000 
#每个孩子有喜欢的100个礼物。(孩子开心)
n_gift_pref = 100 # number of gifts a child ranks
#每个礼物优先考虑1000个好孩子（圣诞老人开心）
n_child_pref = 1000 # number of children a gift ranks
#0-5000为三胞胎，triplets=50001 0.5%
triplets = math.ceil(0.005 * n_children / 3.) * 3    
#5001-45000双胞胎，即45000-5001+1=40000
twins = math.ceil(0.04 * n_children / 2.) * 2    
#根据目标函数里面定义的
ratio_gift_happiness = 2
ratio_child_happiness = 2

In [8]:
#非双胞胎和非三胞胎的孩子，因为成pair，因此-6只是占位符
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = [12*(prefer.shape[0] - i), -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 6*score
        else:
            self.prefer_dict[giftid] = [-6, 6*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])

In [9]:
#双胞胎
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [3*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 3*score
        else:
            self.prefer_dict[giftid] = [-6, 3*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])

In [10]:
#三胞胎
class Child_triplet(object):
"""对于三胞胎，三个人要作为整体考虑
param idx:child id 
param prefer1:三胞胎中第一个孩子偏爱的礼物
param prefer2:三胞胎中第二个孩子偏爱的礼物
param prefer3:三胞胎中第三个孩子偏爱的礼物
"""
    def __init__(self, idx, prefer1, prefer2, prefer3):
        
        self.idx = idx
        self.prefer_dict = dict()
        #三胞胎三个人，有三组不同的喜欢的礼物，所以对于三胞胎set(list(prefer1) + list(prefer2) + list(prefer3))肯定大于100
        for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):
            score = 0
            #对于三胞胎中的第一个人，偏爱有各种礼物的得分
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            #对于三胞胎中的第二个人
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            #对于三胞胎中的第三个人
            if p in list(prefer3):
                score += 2*(100 - list(prefer3).index(p))
            else:
                score -= 1
            #这个是在干啥？
            self.prefer_dict[p] = [2*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        #最佳匹配
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 2*score
        else:
            self.prefer_dict[giftid] = [-6, 2*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])

#### 得到一个孩子喜欢的礼物
gift_pref[index]
#### 三胞胎孩子的礼物
Child_triplet(0, gift_pref[0], gift_pref[1], gift_pref[2])
#### 双胞胎孩子的礼物
Child_twin(5001, gift_pref[5001], gift_pref[5002])
#### 普通孩子
Child(45001, gift_pref[45001])


### 小例子解释作者是在干啥
```python
C=[]
C.append(Child_triplet(0, gift_pref[0], gift_pref[1], gift_pref[2]))
C.append(Child_twin(5001, gift_pref[5001], gift_pref[5002]))
C.append(Child(45001, gift_pref[45001]))

#由于index=0的孩子是三胞胎，因此它偏爱的礼物数是大于100的,从下面的几个条语句可以看出，0号孩子也有偏爱0号礼物的。
#虽然它不在0号孩子偏爱的礼物中，但它在1号孩子偏爱的礼物中。
print(np.where(gift_pref[0]==0))
print(np.where(gift_pref[1]==0))
print(gift_pref[1][40])
print(np.where(gift_pref[2]==0))
(array([], dtype=int64),)
(array([40]),)
0
(array([], dtype=int64),)
#第0个孩子匹配第0个礼物的得分（这样存储的有个问题如果没有礼物则会直接报错）
C[0].prefer_dict[0]
[236, -6]
```

In [33]:
Children = []
for i in range(0, 5001, 3):
    Children.append(Child_triplet(i, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+1, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+2, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
for i in range(5001, 45001, 2):
    Children.append(Child_twin(i, gift_pref[i], gift_pref[i+1]))
    Children.append(Child_twin(i+1, gift_pref[i], gift_pref[i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(45001, 1000000)]

In [38]:
for j in range(1000):
    #每类礼物更偏爱的孩子
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        #同一礼物如果是三胞胎则以三胞胎中最先出现的为主。
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            Children[cid3].add_gifts_prefer(j, score_)
            #如果是双胞胎
            #以双胞胎最先出现的为主
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 1:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            #如果是普通孩子
        elif cf[i] > 45000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

### ######下面还没看，上面的应该都看懂了########

In [39]:
from ortools.graph import pywrapgraph

In [40]:
W_CHILD = 10000
W_GIFTS = 0

In [41]:
min_cost_flow_1 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


In [ ]:


# triplets
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# triplets
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


# add Supply
for i in range(1000):
    min_cost_flow_1.SetNodeSupply(int(1000000+i), int(1000))

# children
for i in range(0, 5001, 3):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))

min_cost_flow_1.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow_1.SetNodeSupply(int(1001000), int(-1000000)) 

#### 求最大公约数math.gcd
```python
greatest common divisor of x and y
math.gcd(319,377)
29
```
- 最大公约数=A×B/最小公倍数,下面的函数实际上是求的最小公倍数
```python
def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)
```


In [35]:
def lcm(a, b):
    """Compute the lowest common multiple of a and b，求的是最小公倍数"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)

In [ ]:

def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if triplets have the same gift
    for t1 in np.arange(0,triplets,3):
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1[1] == triplet1[1] and triplet2[1] == triplet3[1]
                
    # check if twins have the same gift
    for t1 in np.arange(triplets,triplets+twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        # print(t1)
        assert twin1[1] == twin2[1]

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))

    # to avoid float rounding error
    # find common denominator
    # NOTE: I used this code to experiment different parameters, so it was necessary to get the multiplier
    # Note: You should hard-code the multipler to speed up, now that the parameters are finalized
    denominator1 = n_children*max_child_happiness
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type
    common_denom = lcm(denominator1, denominator2)
    multiplier = common_denom / denominator1

    # # usually denom1 > demon2
    return float(math.pow(total_child_happiness*multiplier,3) + math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    # return math.pow(float(total_child_happiness)/(float(n_children)*float(max_child_happiness)),2) + math.pow(np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity),2)

In [4]:
from ortools.graph import pywrapgraph

W_CHILD = 9020
W_GIFTS = 2

In [5]:
min_cost_flow_1 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# triplets
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# triplets
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


# add Supply
for i in range(1000):
    min_cost_flow_1.SetNodeSupply(int(1000000+i), int(1000))

# children
for i in range(0, 5001, 3):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))

min_cost_flow_1.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow_1.SetNodeSupply(int(1001000), int(-1000000)) 




In [6]:
min_cost_flow_1.Solve()

1

In [7]:
assignment = [-1]*1000000
twins_differ = []
triplets_differ = []

for i in range(min_cost_flow_1.NumArcs()):
    if min_cost_flow_1.Flow(i) != 0 and min_cost_flow_1.Head(i) < 1000000:
        c = min_cost_flow_1.Head(i)
        g = min_cost_flow_1.Tail(i)
        f = min_cost_flow_1.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

9020 2 11739596220 1709307 1617929073662152820107354405443
0.9363015472581903


In [8]:
len(twins_differ), len(triplets_differ)

(63, 2)

In [9]:
# well assigned twins and triplets
well_assigned = []
for i in range(0, 5001, 3):
    if assignment[i] == assignment[i+1] and assignment[i] == assignment[i+2]:
        well_assigned.append(i)
        well_assigned.append(i+1)
        well_assigned.append(i+2)
for i in range(5001, 45001, 2):
    if assignment[i] == assignment[i+1]:
        well_assigned.append(i)
        well_assigned.append(i+1)

In [10]:
len(well_assigned)

44869

In [11]:
Gifts_left = [1000 for _ in range(1000)]
for i in well_assigned:
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1

In [12]:
sum(Gifts_left)

958818

In [13]:
# add penalty for twins and triplets
W_CHILD = 90200000
W_GIFTS = 20000

In [14]:
min_cost_flow_2 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# triplets
for i in range(0, 5001, 3):
    if i not in well_assigned:
        for g in Children[i].prefer_dict.keys():
            start_nodes.append(1000000+g)
            end_nodes.append(i)
            capacities.append(3)
            unit_costs.append(1-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
            
# twins
for i in range(5001, 45001, 2):
    if i not in well_assigned:
        for g in Children[i].prefer_dict.keys():
            start_nodes.append(1000000+g)
            end_nodes.append(i)
            capacities.append(2)
            unit_costs.append(1-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


# add Supply
for i in range(1000):
    min_cost_flow_2.SetNodeSupply(int(1000000+i), int(Gifts_left[i]))

# children
for i in range(0, 5001, 3):
    min_cost_flow_2.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_2.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_2.SetNodeSupply(int(i), int(0))

min_cost_flow_2.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow_2.SetNodeSupply(int(1001000), int(-sum(Gifts_left))) 

In [15]:
min_cost_flow_2.Solve()

1

In [16]:
assignment_0 = assignment.copy()

In [17]:
assignment = [-1]*1000000
# pre-assigned
for i in well_assigned:
    assignment[i] = assignment_0[i]
    
twins_differ = []
triplets_differ = []

for i in range(min_cost_flow_2.NumArcs()):
    if min_cost_flow_2.Flow(i) != 0 and min_cost_flow_2.Head(i) < 1000000:
        c = min_cost_flow_2.Head(i)
        g = min_cost_flow_2.Tail(i)
        f = min_cost_flow_2.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

90200000 20000 11739596220 1709307 1617929073662152820107354405443
0.9363015472581903


In [18]:
len(twins_differ), len(triplets_differ)

(11, 2)

In [19]:
# well assigned twins and triplets
well_assigned = []
for i in range(0, 5001, 3):
    if assignment[i] == assignment[i+1] and assignment[i] == assignment[i+2]:
        well_assigned.append(i)
        well_assigned.append(i+1)
        well_assigned.append(i+2)
for i in range(5001, 45001, 2):
    if assignment[i] == assignment[i+1]:
        well_assigned.append(i)
        well_assigned.append(i+1)

In [20]:
Gifts_left = [1000 for _ in range(1000)]
for i in well_assigned:
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1

In [21]:
len(well_assigned)

44973

In [22]:
assignment_1 = assignment.copy()

In [23]:
# add penalty for twins and triplets
W_CHILD = 10000
W_GIFTS = 1

In [24]:
min_cost_flow_3 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []

np.random.seed(71)

# triplets
for i in range(0, 5001, 3):
    if i not in well_assigned:
        for g in Children[i].prefer_dict.keys():
            start_nodes.append(1000000+g)
            end_nodes.append(i)
            capacities.append(3)
            unit_costs.append(np.random.choice(100)-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
            
# twins
for i in range(5001, 45001, 2):
    if i not in well_assigned:
        for g in Children[i].prefer_dict.keys():
            start_nodes.append(1000000+g)
            end_nodes.append(i)
            capacities.append(2)
            unit_costs.append(np.random.choice(100)-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(np.random.choice(5)-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_3.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


# add Supply
for i in range(1000):
    min_cost_flow_3.SetNodeSupply(int(1000000+i), int(Gifts_left[i]))

# children
for i in range(0, 5001, 3):
    min_cost_flow_3.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_3.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_3.SetNodeSupply(int(i), int(0))

min_cost_flow_3.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow_3.SetNodeSupply(int(1001000), int(-sum(Gifts_left))) 

In [25]:
min_cost_flow_3.Solve()

1

In [26]:
assignment = [-1]*1000000
# pre-assigned
for i in well_assigned:
    assignment[i] = assignment_1[i]
    
twins_differ = []
triplets_differ = []

for i in range(min_cost_flow_3.NumArcs()):
    if min_cost_flow_3.Flow(i) != 0 and min_cost_flow_3.Head(i) < 1000000:
        c = min_cost_flow_3.Head(i)
        g = min_cost_flow_3.Tail(i)
        f = min_cost_flow_3.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

10000 1 11739596220 1709307 1617929073662152820107354405443
0.9363015472581903


In [27]:
len(twins_differ), len(triplets_differ)

(1, 0)

In [28]:
twins_differ

[[34267, 34268]]

In [45]:
# well assigned twins and triplets
well_assigned = []
for i in range(0, 5001, 3):
    if assignment[i] == assignment[i+1] and assignment[i] == assignment[i+2]:
        well_assigned.append(i)
        well_assigned.append(i+1)
        well_assigned.append(i+2)
for i in range(5001, 45001, 2):
    if assignment[i] == assignment[i+1]:
        well_assigned.append(i)
        well_assigned.append(i+1)

In [47]:
assignment[34267], assignment[34268]

(207, 494)

In [48]:
assignment_2 = assignment.copy()

In [49]:
# assign 207
assignment[34267] = 207
assignment[34268] = 207

In [50]:
Gifts_left = [1000 for _ in range(1000)]
for i in well_assigned:
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
Gifts_left[207] -= 2

In [51]:
# add penalty for twins and triplets
W_CHILD = 10000
W_GIFTS = 1

In [52]:
# without twins
min_cost_flow_34267 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
      
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_34267.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(45001, 1000000):
    min_cost_flow_34267.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_34267.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_34267.AddArcWithCapacityAndUnitCost(
    int(1001001), int(1001000), int(1000000), int(0)
)


# children
for i in range(45001, 1000000):
    min_cost_flow_34267.SetNodeSupply(int(i), int(0))

min_cost_flow_34267.SetNodeSupply(int(1001001), int(0))





In [53]:
# add Supply
for i in range(1000):
    min_cost_flow_34267.SetNodeSupply(int(1000000+i), int(Gifts_left[i]))
    
min_cost_flow_34267.SetNodeSupply(int(1001000), int(-sum(Gifts_left)))

In [54]:
min_cost_flow_34267.Solve()

1

In [55]:
assignment = [-1]*1000000
# pre-assigned
for i in well_assigned:
    assignment[i] = assignment_2[i]

assignment[34267] = 207
assignment[34268] = 207

for i in range(min_cost_flow_34267.NumArcs()):
    if min_cost_flow_34267.Flow(i) != 0 and min_cost_flow_34267.Head(i) < 1000000:
        c = min_cost_flow_34267.Head(i)
        g = min_cost_flow_34267.Tail(i)

        assignment[c] = g - 1000000

CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

10000 1 11739596220 1703388 1617929073662101118387373555072
0.9363015472581604


In [56]:
unassigned_v = [0, 0, 0]
unassigned_v_ = [0, 0]
for i in range(5001):
    if assignment[i] == -1:
        unassigned_v[0] += 1
for i in range(5001, 45001):
    if assignment[i] == -1:
        unassigned_v[1] += 1
for i in range(45001, 1000000):
    if assignment[i] == -1:
        unassigned_v[2] += 1 
for i in range(0, 5001, 3):
    if assignment[i] == -1:
        unassigned_v_[0] += 1
for i in range(5001, 45001, 2):
    if assignment[i] == -1:
        unassigned_v_[1] += 1
unassigned_v, unassigned_v_

([2742, 950, 0], [914, 475])

In [57]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 628
240 180
272 141
320 496
389 148
494 983
671 707
998 409


In [58]:
gifts_for_trips = [628, 180, 141, 496, 148, 983, 707, 409]
gifts_for_twins = [4, 0, 0, 4, 4, 932, 2, 4]
for i in range(8):
    gifts_for_trips[i] -= gifts_for_twins[i]

In [59]:
gifts_left_list = []
for i in range(gifts_for_trips[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_trips[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_trips[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_trips[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_trips[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_trips[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_trips[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_trips[7]):
    gifts_left_list.append(998)
    
for i in range(gifts_for_twins[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_twins[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_twins[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_twins[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_twins[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_twins[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_twins[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_twins[7]):
    gifts_left_list.append(998)

In [60]:
assignment_3 = assignment.copy()

In [61]:
ind = 0
for i in range(1000000):
    if assignment[i] == -1:
        assignment[i] = gifts_left_list[ind]
        ind += 1

In [62]:
out = open('../submit/seed71_solution.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(1000000):
    out.write(str(i) + ',' + str(assignment[i]) + '\n')
out.close()